# Cumulative players dataframes

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y fielders. Se divide en las siguientes secciones:

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Construcción de variables para estimaciones.**
- **Unión de las bases de datos para nuevas bases transversales.**
- **Segmentación de bases datos de acuerdo a la agencia libre.**
- **Realización del panel data de acuerdo a las especificaciones.**
- **Generación de las variables para el modelo dinámico.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import ListedColormap
from termcolor import colored
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')

In [3]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/ETL_Scripts/Players/Free_Agent_Change


In [4]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

### Equipos por estado

In [5]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [6]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [7]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [8]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [9]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [10]:
acronym_state.head()

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD


En este caso, el nombre de las variables es claro

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [11]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Statistics/Cumulative/Hitting/hitting_'
pitching = 'Data/Statistics/Cumulative/Pitching/pitching_'
salary = 'Data/Salary/salary_'
teams = 'ETL_Data/Transversal/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [12]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [13]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [14]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [15]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'YRS', 'Value', 'AAV'], dtype='object')

In [16]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [17]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS'],
      dtype='object')

In [18]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [19]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                                'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                                'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato',
                                'YRS':'Anios_de_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [20]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(213, 6)
(208, 6)
(221, 6)
(241, 6)
(100, 6)
(98, 6)
(105, 6)
(118, 6)
(141, 6)
(137, 6)


También el tipo de datos que contiene cada columna

In [21]:
df_free_agents_copy[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  100 non-null    object
 1   Status_agente_libre      100 non-null    object
 2   Equipo_anterior          100 non-null    object
 3   Anios_de_contrato        100 non-null    int64 
 4   Valor_contrato           100 non-null    int64 
 5   Valor_promedio_contrato  100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [22]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [23]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [24]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [25]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [26]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [27]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [28]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-9
2
-7


In [29]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [30]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [31]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [32]:
for year in range(0,period):
    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [33]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   object 
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [34]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At-bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs-batted-in',
                            'HR':'Home-runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On-base-plus-slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on-base'})

In [35]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(521, 17)
(669, 17)
(734, 17)
(778, 17)
(809, 17)
(834, 17)
(810, 17)
(825, 17)
(850, 17)
(451, 17)
(753, 17)
(496, 17)


In [36]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    int64  
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    int64  
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At-bats                           834 non-null    int64  
 6   Bateos                            834 non-null    int64  
 7   Dobles                            834 non-null    int64  
 8   Triples                           834 non-null    int64  
 9   Home-runs                         834 non-null    int64  
 10  Runs-batted-in                    834 non-null    int64  
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [37]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

#### Pitchers

In [38]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos', 'R':'Carreras', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike-outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [39]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 16)
(427, 16)
(485, 16)
(537, 16)
(586, 16)
(592, 16)
(581, 16)
(644, 16)
(668, 16)
(542, 16)
(611, 16)
(598, 16)


In [40]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           592 non-null    object 
 1   Juegos            592 non-null    int64  
 2   Juegos_iniciados  592 non-null    int64  
 3   Inning_pitched    592 non-null    float64
 4   Bateos            592 non-null    int64  
 5   Carreras          592 non-null    int64  
 6   Carreras_ganadas  592 non-null    int64  
 7   Walks             592 non-null    int64  
 8   Strike-outs       592 non-null    int64  
 9   Wins              592 non-null    int64  
 10  Losses            592 non-null    int64  
 11  Saves             592 non-null    int64  
 12  WHIP              592 non-null    float64
 13  ERA               592 non-null    float64
 14  WAR               587 non-null    float64
 15  TVS               592 non-null    float64
dtypes: float64(5), int64(10), object(1)
memory u

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $Strike-outs/(Inning \; Pitched)$
- CONTROL = $Walks/(Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [41]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike-outs']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike-outs']/df_pitching_copy[year]['Walks']

In [42]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 19)
(427, 19)
(485, 19)
(537, 19)
(586, 19)
(592, 19)
(581, 19)
(644, 19)
(668, 19)
(542, 19)
(611, 19)
(598, 19)


In [43]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           485 non-null    object 
 1   Juegos            485 non-null    int64  
 2   Juegos_iniciados  485 non-null    int64  
 3   Inning_pitched    485 non-null    float64
 4   Bateos            485 non-null    int64  
 5   Carreras          485 non-null    int64  
 6   Carreras_ganadas  485 non-null    int64  
 7   Walks             485 non-null    int64  
 8   Strike-outs       485 non-null    int64  
 9   Wins              485 non-null    int64  
 10  Losses            485 non-null    int64  
 11  Saves             485 non-null    int64  
 12  WHIP              485 non-null    float64
 13  ERA               485 non-null    float64
 14  WAR               435 non-null    float64
 15  TVS               485 non-null    float64
 16  Dominio           484 non-null    float64
 1

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [44]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Recordemos que el modelo dinámico supone que la compensación salarial está normalizada entre 0 y 1. Por lo que se normalizarán los logaritmos de la compensación, así como las compensacioes en nómina/regulares.

In [45]:
# Define the columns to normalize
comp_to_normalize = ['Sueldo_base', 'Sueldo_ajustado',
                     'Sueldo_regular']

In [46]:
# For cyle for all years:
for year in range(0,period):
    # For cycle for all compensation variables
    for comp in comp_to_normalize:
        new_name = comp + '_norm'
        df_salary_copy[year][new_name] = df_salary_copy[year][comp]

In [47]:
# Define the columns to normalize
normal_compensation = ['Sueldo_base_norm', 'Sueldo_ajustado_norm',
                       'Sueldo_regular_norm']

In [48]:
# For cycle to normalize
for year in range(0,period):
    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Normalize the columns using the fit_transform() method
    df_salary_copy[year][normal_compensation] = scaler.fit_transform(df_salary_copy[year][normal_compensation])

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [49]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [50]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1013 non-null   object 
 1   Anio                     1013 non-null   object 
 2   Posicion                 1013 non-null   object 
 3   Acronimo                 1013 non-null   object 
 4   Sueldo_base              1013 non-null   int64  
 5   Bono_por_firma           1013 non-null   int64  
 6   Sueldo_regular           1013 non-null   int64  
 7   Sueldo_ajustado          1013 non-null   int64  
 8   Sueldo_porcentual        1013 non-null   float64
 9   Pago_efectivo            1013 non-null   int64  
 10  Valor_contrato_promedio  1013 non-null   int64  
 11  Anios_de_contrato        1013 non-null   int64  
 12  Valor_del_contrato       1013 non-null   int64  
 13  Ganancias                1013 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [51]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
    # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)
    
    # Hitters
    mean_war = df_hitting_copy[year].loc[df_hitting_copy[year]['WAR'] > 0].WAR.mean()
    df_hitting_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_hitting_copy[year]['WAR'].mask(df_hitting_copy[year]['WAR'] <= 0, mean_war, inplace = True)

In [52]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike-outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike-outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [53]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike-outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike-outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [54]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [55]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [56]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [57]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
55  -inf
166  -inf
274  -inf

2022
193  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [58]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [59]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [60]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.785784770849762nRegular: 13.888874386883515
Ajustado: 13.81181498978195nRegular: 13.90039782504837
Ajustado: 13.54819591748778nRegular: 13.978975725734408
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575363447521962nRegular: 14.079742661574734
Ajustado: 13.562311948859815nRegular: 14.080756222584538
Ajustado: 13.521364643693584nRegular: 14.066559580453122
Ajustado: 13.029458522392627nRegular: 14.216780831796356
Ajustado: 13.70981799580226nRegular: 14.198804602895482
Ajustado: 13.924314685157658nRegular: 14.351734783233747


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [61]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

In [62]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
       'Carreras', 'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins',
       'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
       'Comando'],
      dtype='object')

In [63]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [64]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                  'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                  'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                  'Porcentaje_On-base-plus-slugging', 'WAR']	
pitching_names = ['Inning_pitched', 'Bateos', 'Carreras',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [65]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
Pitchers:
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3

In [66]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [67]:
for year in range(0,period):
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    # Pitchers:
    for pitcher_name in pitching_indexes:
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

Apreciemos el resultado final

In [68]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             734 non-null    object 
 1   Juegos                              734 non-null    int64  
 2   Porcentaje_juegos                   734 non-null    float64
 3   Juegos_iniciados                    734 non-null    int64  
 4   Porcentaje_juegos_iniciados         734 non-null    float64
 5   At-bats                             734 non-null    int64  
 6   Bateos                              734 non-null    int64  
 7   Dobles                              734 non-null    int64  
 8   Triples                             734 non-null    int64  
 9   Home-runs                           734 non-null    int64  
 10  Runs-batted-in                      734 non-null    int64  
 11  Bateos_promedio                     734 non-n

In [69]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             485 non-null    object 
 1   Juegos              485 non-null    int64  
 2   Juegos_iniciados    485 non-null    int64  
 3   Inning_pitched      485 non-null    float64
 4   Bateos              485 non-null    int64  
 5   Carreras            485 non-null    int64  
 6   Carreras_ganadas    485 non-null    int64  
 7   Walks               485 non-null    int64  
 8   Strike-outs         485 non-null    int64  
 9   Wins                485 non-null    int64  
 10  Losses              485 non-null    int64  
 11  Saves               485 non-null    int64  
 12  WHIP                485 non-null    float64
 13  ERA                 485 non-null    float64
 14  WAR                 485 non-null    float64
 15  TVS                 485 non-null    float64
 16  Dominio 

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [70]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [71]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [72]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [73]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [74]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   object 
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [75]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [76]:
df_salary_copy[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   40 non-null     object 
 1   Cantidad_agentes_libres  40 non-null     int64  
 2   Valor_contrato_total     40 non-null     int64  
 3   Acronimo                 40 non-null     object 
 4   Victorias                40 non-null     int64  
 5   Juegos totales           40 non-null     int64  
 6   Playoffs                 40 non-null     int64  
 7   Pennants won             40 non-null     int64  
 8   WS ganadas               40 non-null     int64  
 9   Promedio_victorias       40 non-null     float64
 10  Estado                   40 non-null     object 
 11  Cantidad de equipos      40 non-null     int64  
 12  Jugador                  40 non-null     object 
 13  Anio                     40 non-null     object 
 14  Posicion                 40 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [77]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [78]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [79]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667 entries, 0 to 666
Data columns (total 67 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             667 non-null    object 
 1   Juegos                              667 non-null    int64  
 2   Porcentaje_juegos                   667 non-null    float64
 3   Juegos_iniciados                    667 non-null    int64  
 4   Porcentaje_juegos_iniciados         667 non-null    float64
 5   At-bats                             667 non-null    int64  
 6   Bateos                              667 non-null    int64  
 7   Dobles                              667 non-null    int64  
 8   Triples                             667 non-null    int64  
 9   Home-runs                           667 non-null    int64  
 10  Runs-batted-in                      667 non-null    int64  
 11  Bateos_promedio                     667 non-n

In [80]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 496
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  497 non-null    object 
 1   Juegos                   497 non-null    int64  
 2   Juegos_iniciados         497 non-null    int64  
 3   Inning_pitched           497 non-null    float64
 4   Bateos                   497 non-null    int64  
 5   Carreras                 497 non-null    int64  
 6   Carreras_ganadas         497 non-null    int64  
 7   Walks                    497 non-null    int64  
 8   Strike-outs              497 non-null    int64  
 9   Wins                     497 non-null    int64  
 10  Losses                   497 non-null    int64  
 11  Saves                    497 non-null    int64  
 12  WHIP                     497 non-null    float64
 13  ERA                      497 non-null    float64
 14  WAR                      4

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [81]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Variables del periodo t-1

Lo que haremos será un *merge* de las bases de datos del año $t$ con el año $t-1$ sobre los jugadores. La razón de esto es que solo nos interesan los jugadores que han sido agentes libres por más de un año.

Si la primera base de datos es del año 2011, entonces tendremos que empezar en el año 2012. Creemos los dataframes que contendrán los datos para el modelo. Para que no se sobrepongan todos los periodos, crearemos dataframes auxiliares para guardar los nuevos datos

In [82]:
hitting_merge = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                 'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                 'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                 'Porcentaje_On-base-plus-slugging', 'WAR',
                 'Juegos_iniciados_2', 'Porcentaje_juegos_iniciados_2', 'At-bats_2', 'Bateos_2',
                 'Dobles_2', 'Triples_2', 'Home-runs_2', 'Runs-batted-in_2', 'Bateos_promedio_2',
                 'Porcentaje_on-base_2', 'Porcentaje_slugging_2', 'TVS_2',
                 'Porcentaje_On-base-plus-slugging_2', 'WAR_2']	
pitching_merge = ['Inning_pitched', 'Bateos_en_contra', 'Carreras_en_contra',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando',
                  'Inning_pitched_2', 'Bateos_2', 'Carreras_2',
                  'Carreras_ganadas_2', 'Walks_2', 'Strike-outs_2', 'Wins_2', 'Losses_2',
                  'Saves_2', 'WHIP_2', 'ERA_2', 'WAR_2', 'TVS_2', 'Dominio_2', 'Control_2',
                  'Comando_2']

In [83]:
df_hitters_copy = [None]*period
df_pitchers_copy = [None]*period

In [84]:
for year in range(0,period):
    df_hitters_copy[year] = df_hitting_copy[year].copy()
    df_pitchers_copy[year] = df_pitching_copy[year].copy()

In [85]:
for year in range(1,period):    
    df_hitting_copy[year] = pd.merge(df_hitters_copy[year], df_hitters_copy[year-1], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitchers_copy[year], df_pitchers_copy[year-1], on = 'Jugador')

A continuación se verifica que la cantidad de columnas sea la misma, salvo por el primer periodo

In [86]:
"""for name in df_pitching_copy[11].columns:
    print(name)"""

'for name in df_pitching_copy[11].columns:\n    print(name)'

In [87]:
for year in range(0,period):
    print(df_hitting_copy[year].columns.shape)
    
for year in range(0,period):    
    print(df_pitching_copy[year].columns.shape)

(67,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(72,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)


In [88]:
for year in range(1,period):       
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_x', '_t')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_y', '_t_1')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('-', '_')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace(' ', '_')
    df_pitching_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                           axis = 1, inplace = True)
    df_pitching_copy[year] = df_pitching_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_x', '_t')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_y', '_t_1')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('-', '_')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace(' ', '_')
    df_hitting_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                          axis = 1, inplace = True)
    df_hitting_copy[year] = df_hitting_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)

In [89]:
for name in df_pitching_copy[11].columns:
    print(name)

Acronimo_t
Acronimo_t_1
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anio_t_1
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Carreras_2_t
Carreras_2_t_1
Carreras_ganadas_2_t
Carreras_ganadas_2_t_1
Carreras_ganadas_t
Carreras_ganadas_t_1
Carreras_t
Carreras_t_1
Comando_2_t
Comando_2_t_1
Comando_t
Comando_t_1
Control_2_t
Control_2_t_1
Control_t
Control_t_1
Dominio_2_t
Dominio_2_t_1
Dominio_t
Dominio_t_1
ERA_2_t
ERA_2_t_1
ERA_t
ERA_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Edad_t_1
Equipo_t
Equipo_t_1
Estado_t
Estado_t_1
Ganancias_t
Ganancias_t_1
Inning_pitched_2_t
Inning_pitched_2_t_1
Inning_pitched_t
Inning_pitched_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Losses_2_t
Losses_2_t_1
Losses_t

Debido a que muchas de las variables del periodo $t_1$ pueden funcionar como controles más realistas, se optarán por dejarlas a excepción de la columna que contiene el dato del año al que hace referencia el dataframe del periodo $t_1$, es decir, la columna *Anio_t_1*. Esto se hará para *pitchers* y *hitters*. Por razones análogas, también de omitirá la columna que indica la cantidad de equipos en determinado estado ya que en el periodo de análisis es invariante.

Para facilitar la escritura del código, entenderemos la columna *Anio* como la columna *Anio_t*.

In [90]:
for year in range(1,period):
    df_pitching_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    df_hitting_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando índice
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)

Cambiemos el súfijo de las basses de datos del año del 2011

In [91]:
year = 0
# Reiniciando los índices
df_hitting_copy[year] = df_hitting_copy[year].add_suffix('_t')
df_pitching_copy[year] = df_pitching_copy[year].add_suffix('_t')
# Corrección de columna del jugador
df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('Jugador_t', 'Jugador')
df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('Jugador_t', 'Jugador')

In [92]:
print("Salarios")
print(df_salary_copy[year].columns)
print("\n")
print("Hitters")
print(df_hitting_copy[year].columns)
print("\n")
print("Pitchers")
print(df_pitching_copy[year].columns)
print("\n")
print("Free agents")
print(df_free_agents_copy[year].columns)
print("\n")

Salarios
Index(['Acronimo', 'Altura', 'Anio', 'Anio_de_agente_libre',
       'Anios_de_contrato', 'Antiguedad', 'Bono_por_firma',
       'Cantidad de equipos', 'Cantidad_agentes_libres', 'Edad',
       'Edad_al_firmar', 'Equipo', 'Estado', 'Ganancias', 'Juegos totales',
       'Jugador', 'Pago_efectivo', 'Pennants won', 'Peso', 'Playoffs',
       'Posicion', 'Promedio_victorias', 'Sueldo_ajustado',
       'Sueldo_ajustado_norm', 'Sueldo_base', 'Sueldo_base_norm',
       'Sueldo_porcentual', 'Sueldo_regular', 'Sueldo_regular_norm',
       'Valor_contrato_promedio', 'Valor_contrato_total', 'Valor_del_contrato',
       'Victorias', 'WS ganadas', 'ln_Sueldo_ajustado', 'ln_Sueldo_base',
       'ln_Sueldo_regular'],
      dtype='object')


Hitters
Index(['Acronimo_t', 'Altura_t', 'Anio_t', 'Anio_de_agente_libre_t',
       'Anios_de_contrato_t', 'Antiguedad_t', 'At-bats_t', 'At-bats_2_t',
       'Bateos_t', 'Bateos_2_t', 'Bateos_promedio_t', 'Bateos_promedio_2_t',
       'Bono_por_firma_t', '

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [93]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [94]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 37
Columns: 132 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(46), int64(75), object(11)
memory usage: 39.5+ KB

 FA - Pitchers:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Columns: 142 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(50), int64(81), object(11)
memory usage: 51.4+ KB

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 278
Columns: 127 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(46), int64(72), object(9)
memory usage: 241.0+ KB

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 353
Columns: 137 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(50), int64(78), object(9)
memory usage: 333.1+ KB


In [95]:
print("FA - Hitters:")
for year in range(0,period):
    print(df_hitters_free_agents[year].shape)
print("\n FA - Pitchers:")
for year in range(0,period):
    print(df_pitchers_free_agents[year].shape)

FA - Hitters:
(0, 72)
(0, 132)
(116, 132)
(61, 132)
(63, 132)
(79, 132)
(45, 132)
(42, 132)
(48, 132)
(38, 132)
(29, 132)
(49, 132)

 FA - Pitchers:
(0, 77)
(2, 142)
(75, 142)
(49, 142)
(53, 142)
(63, 142)
(33, 142)
(44, 142)
(39, 142)
(46, 142)
(46, 142)
(92, 142)


Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [96]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns = str.lower)
    df_pitchers_free_agents[year].rename(columns = str.lower)
    df_hitters_no_free_agents[year].rename(columns = str.lower)
    df_pitchers_no_free_agents[year].rename(columns = str.lower)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

In [97]:
for year in range(0,period):
    # Condiciones
    condicion_hitter = [df_hitting_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    condicion_pitcher = [df_pitching_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    
    # Etiquetas
    etiquetas = ['Si']
    
    df_hitting_copy[year]['Agente_libre'] = np.select(condicion_hitter, etiquetas, default = 'No')
    df_pitching_copy[year]['Agente_libre'] = np.select(condicion_pitcher, etiquetas, default = 'No')
    
    df_hitting_copy[year] = df_hitting_copy[year].reindex(sorted(df_hitting_copy[year].columns), axis=1)
    df_pitching_copy[year] = df_pitching_copy[year].reindex(sorted(df_pitching_copy[year].columns), axis=1)

In [98]:
for year in range(0,period):
    df_hitting_copy[year].rename(columns = str.lower)
    df_pitching_copy[year].rename(columns = str.lower)

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [99]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Filtremos aquellos que no son agentes libres

In [100]:
df_panel_nofa_hitter = df_panel_all_hitter.loc[df_panel_all_hitter['Agente_libre'] == 'No'] 
df_panel_nofa_fielder = df_panel_all_pitcher.loc[df_panel_all_pitcher['Agente_libre'] == 'No'] 

In [101]:
df_panel_nofa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4011 entries, 0 to 291
Columns: 142 entries, Acronimo_t to WS_ganadas_t_1
dtypes: float64(106), int64(26), object(10)
memory usage: 4.4+ MB


In [102]:
df_panel_nofa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3351 entries, 0 to 394
Columns: 144 entries, Acronimo_t to Wins_t_1
dtypes: float64(101), int64(33), object(10)
memory usage: 3.7+ MB


In [103]:
df_panel_fa_hitter = df_panel_all_hitter.loc[df_panel_all_hitter['Agente_libre'] == 'Si'] 
df_panel_fa_fielder = df_panel_all_pitcher.loc[df_panel_all_pitcher['Agente_libre'] == 'Si'] 

In [104]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 4 to 284
Columns: 142 entries, Acronimo_t to WS_ganadas_t_1
dtypes: float64(106), int64(26), object(10)
memory usage: 632.3+ KB


In [105]:
df_panel_fa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541 entries, 6 to 395
Columns: 144 entries, Acronimo_t to Wins_t_1
dtypes: float64(101), int64(33), object(10)
memory usage: 612.9+ KB


In [106]:
df_panel_nofa_hitter.sort_index(axis = 1, inplace = True)
df_panel_nofa_fielder.sort_index(axis = 1, inplace = True)
df_panel_fa_hitter.sort_index(axis = 1, inplace = True)
df_panel_fa_fielder.sort_index(axis = 1, inplace = True)

Debido al método que se quiere usar tenemos que asegurarnos de quedarno solo con la última y primer observación de cada jugador de los páneles de arriba, respectivamente.

In [107]:
# No agentes libres:
df_panel_nofa_hitter = df_panel_nofa_hitter.sort_values('Anio_t').groupby('Jugador').tail(1)
df_panel_nofa_fielder = df_panel_nofa_fielder.sort_values('Anio_t').groupby('Jugador').tail(1)

# Agentes libres:
df_panel_fa_hitter = df_panel_fa_hitter.sort_values('Anio_t').groupby('Jugador').first(1)
df_panel_fa_fielder = df_panel_fa_fielder.sort_values('Anio_t').groupby('Jugador').first(1)

# Reindexando
df_panel_fa_hitter = df_panel_fa_hitter.reset_index()
df_panel_fa_fielder = df_panel_fa_fielder.reset_index()

Creemos un dataframe que contengan solo a los jugadores en común entre las bases

In [108]:
# Bateadores:
df_hitter_change = pd.merge(df_panel_nofa_hitter,
                            df_panel_fa_hitter,
                            on = 'Jugador')

# Fildeadores:
df_fielder_change = pd.merge(df_panel_nofa_fielder,
                             df_panel_fa_fielder,
                             on = 'Jugador')

Sin embargo, el merge solo nos sirve para filtrar los nombres de los jugadores que están en ambas bases de datos, así que filtremoslos de los otros páneles

In [109]:
# Names in merge:
hitter_change_names = df_hitter_change['Jugador'].tolist()
fielder_change_names = df_fielder_change['Jugador'].tolist()

# Filtration
df_panel_nofa_hitter = df_panel_nofa_hitter[df_panel_nofa_hitter['Jugador'].isin(hitter_change_names)]
df_panel_nofa_fielder = df_panel_nofa_fielder[df_panel_nofa_fielder['Jugador'].isin(fielder_change_names)]
df_panel_fa_hitter = df_panel_fa_hitter[df_panel_fa_hitter['Jugador'].isin(hitter_change_names)]
df_panel_fa_fielder = df_panel_fa_fielder[df_panel_fa_fielder['Jugador'].isin(fielder_change_names)]

# Reiniciando índice
df_panel_nofa_hitter.reset_index(drop = True,
                                 inplace = True)
df_panel_nofa_fielder.reset_index(drop = True,
                                  inplace = True)
df_panel_fa_hitter.reset_index(drop = True,
                               inplace = True)
df_panel_fa_fielder.reset_index(drop = True,
                                inplace = True)

Agreguemos una columna auxiliar que indique el periodo relativo

In [110]:
df_panel_nofa_hitter['Anio'] = 0
df_panel_nofa_fielder['Anio'] = 0
df_panel_fa_hitter['Anio'] = 1
df_panel_fa_fielder['Anio'] = 1

In [111]:
df_panel_nofa_hitter['Status_agente_libre'] = 'No'
df_panel_nofa_fielder['Status_agente_libre'] = 'No'
df_panel_fa_hitter['Status_agente_libre'] = 'Si'
df_panel_fa_fielder['Status_agente_libre'] = 'Si'

In [112]:
df_panel_nofa_hitter['Equipo_anterior'] = 'No'
df_panel_nofa_fielder['Equipo_anterior'] = 'No'
df_panel_fa_hitter['Equipo_anterior'] = 'Si'
df_panel_fa_fielder['Equipo_anterior'] = 'Si'

Verifiquemos que las dimensiones de los renglones sean las mismas

In [113]:
df_panel_nofa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Columns: 145 entries, Acronimo_t to Equipo_anterior
dtypes: float64(106), int64(27), object(12)
memory usage: 335.4+ KB


In [114]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Columns: 136 entries, Jugador to Equipo_anterior
dtypes: float64(106), int64(27), object(3)
memory usage: 314.6+ KB


In [115]:
df_panel_nofa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Columns: 147 entries, Acronimo_t to Equipo_anterior
dtypes: float64(101), int64(34), object(12)
memory usage: 313.6+ KB


In [116]:
df_panel_fa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Columns: 138 entries, Jugador to Equipo_anterior
dtypes: float64(101), int64(34), object(3)
memory usage: 294.5+ KB


Por último, concatenemos los respectivos páneles

In [117]:
# Hitter
df_hitter_change = pd.concat([df_panel_nofa_hitter,
                              df_panel_fa_hitter])

# Pitcher
df_fielder_change = pd.concat([df_panel_nofa_fielder,
                               df_panel_fa_fielder])

In [118]:
df_hitter_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 295
Columns: 145 entries, Acronimo_t to Equipo_anterior
dtypes: float64(106), int64(27), object(12)
memory usage: 675.2+ KB


In [119]:
df_fielder_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 272
Columns: 147 entries, Acronimo_t to Equipo_anterior
dtypes: float64(101), int64(34), object(12)
memory usage: 631.3+ KB


# Variables del Modelo Empírico

In [120]:
empiric_panel_hitter = df_hitter_change.copy()
empiric_panel_pitcher = df_fielder_change.copy()

Veamos algunas estadísticas e información que contienen las bases de datos

In [121]:
print(empiric_panel_hitter.shape)

(592, 145)


In [122]:
print(empiric_panel_pitcher.shape)

(546, 147)


Las posiciones que hay en cada base de datos

In [123]:
empiric_panel_hitter['Posicion_t'].unique()

array(['RF', '2B', 'C', 'SS', 'SP', 'LF', '1B', 'RP', '3B', 'DH', 'CF',
       'OF', nan], dtype=object)

In [124]:
empiric_panel_pitcher['Posicion_t'].unique()

array(['RP', 'SP', 'RP/CL', 'SS', nan], dtype=object)

Ordenemos las bases de datos  por nombre y año

In [125]:
# Hitter
empiric_panel_hitter = empiric_panel_hitter.sort_values(by = ['Jugador','Anio_t'], ascending=True)
empiric_panel_hitter.reset_index(drop = True, inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher.sort_values(by = ['Jugador','Anio_t'], ascending=True)
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

In [126]:
empiric_panel_hitter[['Jugador','Anio_t']].head()

,Jugador,Anio_t
0,A.J. Burnett,2013
1,A.J. Burnett,NaN
2,A.J. Ellis,2018
3,A.J. Ellis,NaN
4,A.J. Pollock,2022


In [127]:
empiric_panel_pitcher[['Jugador','Anio_t']].head()

,Jugador,Anio_t
0,A.J. Burnett,2013
1,A.J. Burnett,NaN
2,Aaron Harang,2014
3,Aaron Harang,NaN
4,Adam Ottavino,2021


In [128]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

Obtengamos los índices de las columnas de interes

In [129]:
hitter_regular_stats = ['At_bats_2_t_1', 'At_bats_t_1',
                        'Bateos_2_t_1', 'Bateos_t_1',
                        'Bateos_promedio_2_t_1', 'Bateos_promedio_t_1',
                        'Dobles_2_t_1', 'Dobles_t_1',
                        'Home_runs_2_t_1', 'Home_runs_t_1',
                        'Juegos_iniciados_2_t_1', 'Juegos_iniciados_t_1', 
                        'Porcentaje_On_base_plus_slugging_2_t_1', 'Porcentaje_On_base_plus_slugging_t_1',
                        'Porcentaje_on_base_2_t_1', 'Porcentaje_on_base_t_1',
                        'Porcentaje_slugging_2_t_1', 'Porcentaje_slugging_t_1',
                        'Runs_batted_in_2_t_1', 'Runs_batted_in_t_1',
                        'Triples_2_t_1', 'Triples_t_1',
                        'WAR_2_t_1', 'WAR_t_1']
hitter_regular_stats = sorted(hitter_regular_stats)

In [130]:
# Hitter
for stat in range(0,len(hitter_regular_stats)):
    # Variables auxiliares
    stat_name = hitter_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo_t":empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].max().index,
                             max_stat_name: empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo_t":empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].min().index,
                             min_stat_name: empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].min().values})
    empiric_panel_hitter = empiric_panel_hitter.merge(max_stat, on = "Acronimo_t",
                                                      how = "left")
    empiric_panel_hitter = empiric_panel_hitter.merge(min_stat, on = "Acronimo_t",
                                                      how = "left")

In [131]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [132]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_t_1_H')
end = empiric_panel_hitter.columns.get_loc('WAR_t_1_L') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_t_1_H,At_bats_2_t_1_L,At_bats_t_1_H,At_bats_t_1_L,Bateos_2_t_1_H,Bateos_2_t_1_L,Bateos_promedio_2_t_1_H,Bateos_promedio_2_t_1_L,Bateos_promedio_t_1_H,Bateos_promedio_t_1_L,...,Runs_batted_in_t_1_H,Runs_batted_in_t_1_L,Triples_2_t_1_H,Triples_2_t_1_L,Triples_t_1_H,Triples_t_1_L,WAR_2_t_1_H,WAR_2_t_1_L,WAR_t_1_H,WAR_t_1_L
0,239121.0,1.0,489.0,1.0,23104.0,0.0,0.096721,0.0,0.311,0.0,...,86.0,0.0,25.0,0.0,5.0,0.0,23.3289,0.0016,4.83,0.04
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,318096.0,64.0,564.0,8.0,24649.0,0.0,0.082369,0.0,0.287,0.0,...,106.0,0.0,16.0,0.0,4.0,0.0,25.4016,0.0004,5.04,0.02
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,386884.0,1.0,622.0,1.0,28900.0,0.0,0.088804,0.0,0.298,0.0,...,75.0,0.0,4.0,0.0,2.0,0.0,13.9129,0.1369,3.73,0.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
588,241081.0,9.0,491.0,3.0,14400.0,0.0,0.071824,0.0,0.268,0.0,...,93.0,0.0,4.0,0.0,2.0,0.0,5.1529,0.0016,2.27,0.04
589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590,257049.0,9.0,507.0,3.0,21316.0,0.0,0.096721,0.0,0.311,0.0,...,88.0,0.0,9.0,0.0,3.0,0.0,15.7609,0.6561,3.97,0.81


Repitamos el mismo proceso para los lanzadores

In [133]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

Repitamos el mismo proceso para los filderos

In [134]:
pitcher_regular_stats = ['Bateos_2_t_1', 'Bateos_t_1',
                        'Carreras_2_t_1', 'Carreras_t_1',
                        'Carreras_ganadas_2_t_1', 'Carreras_ganadas_t_1',
                        'Comando_2_t_1', 'Comando_t_1',
                        'Control_2_t_1', 'Control_t_1',
                        'Dominio_2_t_1', 'Dominio_t_1', 
                        'ERA_2_t_1', 'ERA_t_1',
                        'Inning_pitched_2_t_1', 'Inning_pitched_t_1',
                        'Losses_2_t_1', 'Losses_t_1',
                        'Saves_2_t_1', 'Saves_t_1',
                        'Strike_outs_2_t_1', 'Strike_outs_t_1',
                        'WAR_2_t_1', 'WAR_t_1',
                        'WHIP_2_t_1', 'WHIP_t_1',
                        'Walks_2_t_1', 'Walks_t_1',
                        'Wins_2_t_1', 'Wins_t_1']
pitcher_regular_stats = sorted(pitcher_regular_stats)

In [135]:
# Pitcheer
for stat in range(0,len(pitcher_regular_stats)):
    # Variables auxiliares
    stat_name = pitcher_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo_t":empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].max().index,
                             max_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo_t":empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].min().index,
                             min_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].min().values})
    empiric_panel_pitcher = empiric_panel_pitcher.merge(max_stat, on = "Acronimo_t",
                                                        how = "left")
    empiric_panel_pitcher = empiric_panel_pitcher.merge(min_stat, on = "Acronimo_t",
                                                        how = "left")

In [136]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [137]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_t_1_H')
end = empiric_panel_pitcher.columns.get_loc('Wins_t_1_L') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_t_1_H,Bateos_2_t_1_L,Bateos_t_1_H,Bateos_t_1_L,Carreras_2_t_1_H,Carreras_2_t_1_L,Carreras_ganadas_2_t_1_H,Carreras_ganadas_2_t_1_L,Carreras_ganadas_t_1_H,Carreras_ganadas_t_1_L,...,WHIP_t_1_H,WHIP_t_1_L,Walks_2_t_1_H,Walks_2_t_1_L,Walks_t_1_H,Walks_t_1_L,Wins_2_t_1_H,Wins_2_t_1_L,Wins_t_1_H,Wins_t_1_L
0,35721.0,361.0,189.0,19.0,7396.0,49.0,6241.0,49.0,79.0,7.0,...,1.52,1.01,5329.0,121.0,73.0,11.0,256.0,4.0,16.0,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31684.0,64.0,178.0,8.0,8281.0,9.0,7396.0,4.0,86.0,2.0,...,1.60,0.61,3364.0,9.0,58.0,3.0,196.0,0.0,14.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,38025.0,400.0,195.0,20.0,9604.0,144.0,8649.0,100.0,93.0,10.0,...,1.58,0.92,5041.0,81.0,71.0,9.0,289.0,1.0,17.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542,29241.0,81.0,171.0,9.0,6561.0,1.0,5625.0,1.0,75.0,1.0,...,1.39,0.91,2500.0,25.0,50.0,5.0,225.0,0.0,15.0,0.0
543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
544,22500.0,16.0,150.0,4.0,7056.0,0.0,6241.0,0.0,79.0,0.0,...,2.06,0.88,3025.0,9.0,55.0,3.0,169.0,0.0,13.0,0.0


In [138]:
empiric_panel_hitter.shape

(592, 193)

In [139]:
empiric_panel_pitcher.shape

(546, 207)

### Filtrando jugadores con más de dos años de observaciones en nuestro panel

Para ser consistentes, las columnas que contienen datos de tipo *string* las imputaremos con la palabra *No* ya que representará que no tenía equipo, ni posición, etc.

In [140]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 591
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t           296 non-null    object
 1   Acronimo_t_1         296 non-null    object
 2   Agente_libre         296 non-null    object
 3   Anio_t               296 non-null    object
 4   Equipo_t             296 non-null    object
 5   Equipo_t_1           296 non-null    object
 6   Estado_t             296 non-null    object
 7   Jugador              592 non-null    object
 8   Posicion_t           296 non-null    object
 9   Posicion_t_1         296 non-null    object
 10  Status_agente_libre  592 non-null    object
 11  Equipo_anterior      592 non-null    object
dtypes: object(12)
memory usage: 60.1+ KB


In [141]:
empiric_panel_hitter[['Acronimo_t',
                      'Equipo_anterior',
                      'Equipo_t',
                      'Estado_t',
                      'Posicion_t',
                      'Status_agente_libre',
                      'Acronimo_t_1',
                      'Equipo_t_1',
                      'Posicion_t_1']] = \
empiric_panel_hitter[['Acronimo_t',
                      'Equipo_anterior',
                      'Equipo_t',
                      'Estado_t',
                      'Posicion_t',
                      'Status_agente_libre',
                      'Acronimo_t_1',
                      'Equipo_t_1',
                      'Posicion_t_1']].fillna('No')

Veamos si funcionó la imputación

In [142]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 592 entries, 0 to 591
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t           592 non-null    object
 1   Acronimo_t_1         592 non-null    object
 2   Agente_libre         296 non-null    object
 3   Anio_t               296 non-null    object
 4   Equipo_t             592 non-null    object
 5   Equipo_t_1           592 non-null    object
 6   Estado_t             592 non-null    object
 7   Jugador              592 non-null    object
 8   Posicion_t           592 non-null    object
 9   Posicion_t_1         592 non-null    object
 10  Status_agente_libre  592 non-null    object
 11  Equipo_anterior      592 non-null    object
dtypes: object(12)
memory usage: 60.1+ KB


Ahora con los lanzadores

In [143]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 545
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t           273 non-null    object
 1   Acronimo_t_1         273 non-null    object
 2   Agente_libre         273 non-null    object
 3   Anio_t               273 non-null    object
 4   Equipo_t             273 non-null    object
 5   Equipo_t_1           273 non-null    object
 6   Estado_t             273 non-null    object
 7   Jugador              546 non-null    object
 8   Posicion_t           273 non-null    object
 9   Posicion_t_1         273 non-null    object
 10  Status_agente_libre  546 non-null    object
 11  Equipo_anterior      546 non-null    object
dtypes: object(12)
memory usage: 55.5+ KB


In [144]:
empiric_panel_pitcher[['Acronimo_t',
                       'Equipo_anterior',
                       'Equipo_t',
                       'Estado_t',
                       'Posicion_t',
                       'Status_agente_libre',
                       'Acronimo_t_1',
                       'Equipo_t_1',
                       'Posicion_t_1']] = \
empiric_panel_pitcher[['Acronimo_t',
                       'Equipo_anterior',
                       'Equipo_t',
                       'Estado_t',
                       'Posicion_t',
                       'Status_agente_libre',
                       'Acronimo_t_1',
                       'Equipo_t_1',
                       'Posicion_t_1']].fillna('No')

In [145]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 546 entries, 0 to 545
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Acronimo_t           546 non-null    object
 1   Acronimo_t_1         546 non-null    object
 2   Agente_libre         273 non-null    object
 3   Anio_t               273 non-null    object
 4   Equipo_t             546 non-null    object
 5   Equipo_t_1           546 non-null    object
 6   Estado_t             546 non-null    object
 7   Jugador              546 non-null    object
 8   Posicion_t           546 non-null    object
 9   Posicion_t_1         546 non-null    object
 10  Status_agente_libre  546 non-null    object
 11  Equipo_anterior      546 non-null    object
dtypes: object(12)
memory usage: 55.5+ KB


En ambos casos, la imputación fue un éxito. No importa que en periodos de contratación de haya puesto *No* ya que con esto evitamos problemas con posibles instrumentos a partir de dummies.

Por otro lado, repitamos lo mismo para las columnas numéricas, imputaremos 0 ya que refleja la ausencia de desempeño. Sin embargo, la columna que contiene la estadística más alta de cada medida de desempeño se imputará por separado debido a que causará problemas cuando se imponga la trnasformación para el modelo.

In [146]:
# Add string to every element in a list
# Hitter
hitter_high_stat = [stat + '_H' for stat in hitter_regular_stats]
hitter_low_stat = [stat + '_L' for stat in hitter_regular_stats]
# Fielder
pitcher_high_stat = [stat + '_H' for stat in pitcher_regular_stats]
pitcher_low_stat = [stat + '_L' for stat in pitcher_regular_stats]

In [147]:
empiric_panel_hitter[hitter_high_stat] = empiric_panel_hitter[hitter_high_stat].fillna(method = 'ffill')
empiric_panel_pitcher[pitcher_high_stat] = empiric_panel_pitcher[pitcher_high_stat].fillna(method = 'ffill')

In [148]:
empiric_panel_hitter[hitter_high_stat].head(20)

,At_bats_2_t_1_H,At_bats_t_1_H,Bateos_2_t_1_H,Bateos_promedio_2_t_1_H,Bateos_promedio_t_1_H,Bateos_t_1_H,Dobles_2_t_1_H,Dobles_t_1_H,Home_runs_2_t_1_H,Home_runs_t_1_H,...,Porcentaje_on_base_2_t_1_H,Porcentaje_on_base_t_1_H,Porcentaje_slugging_2_t_1_H,Porcentaje_slugging_t_1_H,Runs_batted_in_2_t_1_H,Runs_batted_in_t_1_H,Triples_2_t_1_H,Triples_t_1_H,WAR_2_t_1_H,WAR_t_1_H
0,239121.0,489.0,23104.0,0.096721,0.311,152.0,1156.0,34.0,729.0,27.0,...,0.142129,0.377,0.266256,0.516,7396.0,86.0,25.0,5.0,23.3289,4.83
1,239121.0,489.0,23104.0,0.096721,0.311,152.0,1156.0,34.0,729.0,27.0,...,0.142129,0.377,0.266256,0.516,7396.0,86.0,25.0,5.0,23.3289,4.83
2,318096.0,564.0,24649.0,0.082369,0.287,157.0,961.0,31.0,784.0,28.0,...,0.120409,0.347,0.267289,0.517,11236.0,106.0,16.0,4.0,25.4016,5.04
3,318096.0,564.0,24649.0,0.082369,0.287,157.0,961.0,31.0,784.0,28.0,...,0.120409,0.347,0.267289,0.517,11236.0,106.0,16.0,4.0,25.4016,5.04
4,386884.0,622.0,28900.0,0.088804,0.298,170.0,1225.0,35.0,676.0,26.0,...,0.176400,0.420,0.287296,0.536,5625.0,75.0,4.0,2.0,13.9129,3.73
5,386884.0,622.0,28900.0,0.088804,0.298,170.0,1225.0,35.0,676.0,26.0,...,0.176400,0.420,0.287296,0.536,5625.0,75.0,4.0,2.0,13.9129,3.73
6,310249.0,557.0,17689.0,0.116281,0.341,133.0,625.0,25.0,1444.0,38.0,...,0.213444,0.462,0.409600,0.640,12769.0,113.0,9.0,3.0,10.8900,3.30
7,310249.0,557.0,17689.0,0.116281,0.341,133.0,625.0,25.0,1444.0,38.0,...,0.213444,0.462,0.409600,0.640,12769.0,113.0,9.0,3.0,10.8900,3.30
8,310249.0,557.0,17689.0,0.116281,0.341,133.0,625.0,25.0,1444.0,38.0,...,0.213444,0.462,0.409600,0.640,12769.0,113.0,9.0,3.0,10.8900,3.30
9,310249.0,557.0,17689.0,0.116281,0.341,133.0,625.0,25.0,1444.0,38.0,...,0.213444,0.462,0.409600,0.640,12769.0,113.0,9.0,3.0,10.8900,3.30


In [149]:
empiric_panel_hitter.fillna(0, inplace = True)
empiric_panel_pitcher.fillna(0, inplace = True)

Verifiquemos si queda alguna columna con alguna entrada tipo *NaN*:

In [150]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [151]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

Obtengamos el máximo de cada una de las medidas de desempeño, de periodos *t_1*, por equipo que han obtenido a lo largo de todas la temporadas

Para evitar problemas con el tipo de *ID*, creemos una que sea numérica para evitar usar los nombres de los jugadores

In [152]:
# Hitter
empiric_panel_hitter['id'] =  empiric_panel_hitter.groupby(['Jugador']).ngroup()
empiric_panel_hitter.reset_index(drop = True, inplace = True)
# Pitcher
empiric_panel_pitcher['id'] =  empiric_panel_pitcher.groupby(['Jugador']).ngroup()
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

Obtengamos la transformación para obtener la $Y$ del modelo empírico a partir de los rezagos de las raices de dichos salarios

In [153]:
# Función de rezagos de raices
def sqrt_dif(X):
    S = []
    for i in range(0, len(X)-1):
        d = np.sqrt(X[i+1])-np.sqrt(X[i])
        S.append(d)
    try:
        S.append(d)
    except: 
        S.append(0)
    return S

Haremos un ciclo para hacer sobre todas las posibilidades de variables de compensación salarial

In [154]:
compensations = ['Sueldo_ajustado_norm_t', 'Sueldo_regular_norm_t',
                 'Sueldo_base_norm_t', 'TVS_t',
                 'ln_Sueldo_ajustado_t', 'ln_Sueldo_regular_t']

In [155]:
for i in range(len(compensations)):
    Y_hitter = []
    for p in empiric_panel_hitter["id"].unique():
        # Filtremos todos los sueldos (ln) de cada jugador por separado
        X = empiric_panel_hitter[empiric_panel_hitter["id"] == p][compensations[i]].values
        # Aplicación de la función
        S = sqrt_dif(X)
        # Añadimos los datos de manera ordenada
        Y_hitter = np.concatenate((Y_hitter, S))
    # Agregamos la columna:
    Y_name = 'Y_' + compensations[i]
    empiric_panel_hitter[Y_name] = Y_hitter

In [156]:
for i in range(len(compensations)):
    Y_pitcher = []
    for p in empiric_panel_pitcher["id"].unique():
        # Filtremos todos los sueldos (ln) de cada jugador por separado
        X = empiric_panel_pitcher[empiric_panel_pitcher["id"] == p][compensations[i]].values
        # Aplicación de la función
        S = sqrt_dif(X)
        # Añadimos los datos de manera ordenada
        Y_pitcher = np.concatenate((Y_pitcher, S))
    # Agregamos la columna:
    Y_name = 'Y_' + compensations[i]
    empiric_panel_pitcher[Y_name] = Y_pitcher

Contruyamos las dummy *I* del modelo empírico

In [157]:
# Long of the stats
end_hitter_name = int((len(hitter_high_stat) + len(hitter_low_stat))/2)

In [158]:
hitter_regular_stats

['At_bats_2_t_1',
 'At_bats_t_1',
 'Bateos_2_t_1',
 'Bateos_promedio_2_t_1',
 'Bateos_promedio_t_1',
 'Bateos_t_1',
 'Dobles_2_t_1',
 'Dobles_t_1',
 'Home_runs_2_t_1',
 'Home_runs_t_1',
 'Juegos_iniciados_2_t_1',
 'Juegos_iniciados_t_1',
 'Porcentaje_On_base_plus_slugging_2_t_1',
 'Porcentaje_On_base_plus_slugging_t_1',
 'Porcentaje_on_base_2_t_1',
 'Porcentaje_on_base_t_1',
 'Porcentaje_slugging_2_t_1',
 'Porcentaje_slugging_t_1',
 'Runs_batted_in_2_t_1',
 'Runs_batted_in_t_1',
 'Triples_2_t_1',
 'Triples_t_1',
 'WAR_2_t_1',
 'WAR_t_1']

In [159]:
# Hitter
for sport_stat in range(0,end_hitter_name):
    I_hitter = []
    for y,max_stat,min_stat in zip(empiric_panel_hitter[hitter_regular_stats[sport_stat]],
                                   empiric_panel_hitter[hitter_high_stat[sport_stat]],
                                   empiric_panel_hitter[hitter_low_stat[sport_stat]]):
        # Dummy condition
        if y > (max_stat + min_stat)/2:
            I_hitter.append(0)
        else: 
            I_hitter.append(1)
    
    I_name = hitter_regular_stats[sport_stat] + '_I'
    empiric_panel_hitter[I_name] = I_hitter

Veamos los resultados

In [160]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [161]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [162]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_t_1_I')
end = empiric_panel_hitter.columns.get_loc('WAR_t_1_I') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_t_1_I,At_bats_t_1_I,Bateos_2_t_1_I,Bateos_promedio_2_t_1_I,Bateos_promedio_t_1_I,Bateos_t_1_I,Dobles_2_t_1_I,Dobles_t_1_I,Home_runs_2_t_1_I,Home_runs_t_1_I,...,Porcentaje_on_base_2_t_1_I,Porcentaje_on_base_t_1_I,Porcentaje_slugging_2_t_1_I,Porcentaje_slugging_t_1_I,Runs_batted_in_2_t_1_I,Runs_batted_in_t_1_I,Triples_2_t_1_I,Triples_t_1_I,WAR_2_t_1_I,WAR_t_1_I
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,0,0,1,1,1,1,1,...,0,0,0,0,1,1,1,1,1,1
3,1,1,1,0,0,1,1,1,1,1,...,0,0,1,0,1,1,1,1,1,1
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
588,1,1,1,0,0,1,1,0,1,1,...,1,1,0,0,1,1,0,0,1,1
589,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
590,1,1,1,0,0,1,1,1,1,1,...,0,0,1,0,1,1,1,1,0,0


Repitamos el mismo proceso para los lanzadores

In [163]:
# Long of stats
end_pitcher_name = int((len(pitcher_high_stat) + len(pitcher_low_stat))/2)

In [164]:
# Pitcher
for sport_stat in range(0,end_pitcher_name):
    I_pitcher = []
    for y,max_stat,min_stat in zip(empiric_panel_pitcher[pitcher_regular_stats[sport_stat]],
                                   empiric_panel_pitcher[pitcher_high_stat[sport_stat]],
                                   empiric_panel_pitcher[pitcher_low_stat[sport_stat]]):
        if y > (max_stat + min_stat)/2:
            I_pitcher.append(0)
        else: 
            I_pitcher.append(1)
    
    I_name = pitcher_regular_stats[sport_stat] + '_I'
    empiric_panel_pitcher[I_name] = I_pitcher

In [165]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [192]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

In [167]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_t_1_I')
end = empiric_panel_pitcher.columns.get_loc('Wins_t_1_I') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_t_1_I,Bateos_t_1_I,Carreras_2_t_1_I,Carreras_ganadas_2_t_1_I,Carreras_ganadas_t_1_I,Carreras_t_1_I,Comando_2_t_1_I,Comando_t_1_I,Control_2_t_1_I,Control_t_1_I,...,Strike_outs_2_t_1_I,Strike_outs_t_1_I,WAR_2_t_1_I,WAR_t_1_I,WHIP_2_t_1_I,WHIP_t_1_I,Walks_2_t_1_I,Walks_t_1_I,Wins_2_t_1_I,Wins_t_1_I
0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,1,1,1,...,1,1,1,1,0,0,1,0,1,1
3,0,0,0,0,0,0,1,1,1,0,...,0,0,1,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,1,1,1,1,1,1,1,1,1,0,...,1,1,1,1,0,0,1,1,1,1
542,1,1,1,1,1,1,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
543,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
544,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,1,1,0,0,0,0


Obtengamos las variables auxiliares

In [168]:
# Good practice
hitter_high_stat = sorted(hitter_high_stat)
# Dummy
I_hitter = sorted([stat + '_I' for stat in hitter_regular_stats])

Verifiquemos que tengan la misma longitud

In [169]:
len(I_hitter) == len(hitter_high_stat) == len(hitter_regular_stats)

True

In [170]:
# Pitcher
for stat in range(0,len(hitter_regular_stats)):
    # Variable auxiliar
    X_hitter = []
    
    # Variables 
    i = (-1)**(empiric_panel_hitter[I_hitter[stat]])
    x = empiric_panel_hitter[hitter_regular_stats[stat]]/np.sqrt(empiric_panel_hitter[hitter_high_stat[stat]])
    X_hitter = i*x
    
    # X name
    name = 'X_' + hitter_regular_stats[stat]
    empiric_panel_hitter[name] = X_hitter

In [171]:
# Pitcher
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

Name: X_Triples_2_t_1
Name: X_Triples_t_1


In [172]:
# calculate the mean of each column
hitter_means = empiric_panel_hitter.mean()

# impute missing data with the mean value of the column
empiric_panel_hitter = empiric_panel_hitter.fillna(hitter_means)

In [173]:
# define a list of suffixes to drop
drop_suffix = ['_I', '_L', '_H']

# Use a list comprehension to filter the columns to drop
hitter_cols_to_drop = [col for col in empiric_panel_hitter.columns if col.endswith(tuple(drop_suffix))]

# Drop the selected columns
empiric_panel_hitter = empiric_panel_hitter.drop(hitter_cols_to_drop,
                                                 axis=1)

In [174]:
empiric_panel_hitter.shape

(592, 176)

In [175]:
empiric_panel_hitter['Posicion_t'].value_counts()

No    296
SP     61
C      37
LF     35
1B     24
2B     23
RF     21
SS     20
RP     19
CF     19
DH     18
3B     17
OF      2
Name: Posicion_t, dtype: int64

Veamos todas las variables del panel

In [176]:
panel_columns = empiric_panel_hitter.columns

for name in panel_columns:
    print(name)

Acronimo_t
Acronimo_t_1
Agente_libre
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
At-bats_2_t
At-bats_t
At_bats_2_t
At_bats_2_t_1
At_bats_t
At_bats_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_promedio_2_t
Bateos_promedio_2_t_1
Bateos_promedio_t
Bateos_promedio_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad de equipos_t
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Dobles_2_t
Dobles_2_t_1
Dobles_t
Dobles_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Equipo_t
Equipo_t_1
Estado_t
Ganancias_t
Ganancias_t_1
Home-runs_2_t
Home-runs_t
Home_runs_2_t
Home_runs_2_t_1
Home_runs_t
Home_runs_t_1
Juegos totales_t
Juegos_iniciados_2_t
Juegos_iniciados_2_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Pago_efectivo_t
Pago_efectivo_t_1
Pennants won_t
Pennants_won_t
Pennants_won_

In [177]:
general_dynamic_path = 'ETL_Data/Panel/Cumulative/Bargaining_change/'

In [178]:
empiric_panel_hitter.to_csv(general_dynamic_path + 'panel_hitters_cum_ch' + '.csv',
                            index = False)

Ahora, filtremos las columnas que contienen las medidas de desempeño de los periodos **t_1**. Para ello, encontremos primero los índices de dichas variables

In [179]:
Y_list = ['Y_Sueldo_ajustado_norm_t',
          'Y_Sueldo_regular_norm_t',
          'Y_Sueldo_base_norm_t',
          'Y_TVS_t',
          'Y_ln_Sueldo_ajustado_t',
          'Y_ln_Sueldo_regular_t']

In [180]:
# Pitcher
for y_name in Y_list:
    for stat in range(0,len(hitter_regular_stats)):
        # Stat
        print(colored(hitter_regular_stats[stat], "cyan"))

        # X name
        name = 'X_' + hitter_regular_stats[stat]

        # OLS variables
        Y = empiric_panel_hitter[y_name].tolist()
        X = empiric_panel_hitter[name].tolist()
        X = sm.add_constant(X)

        # Modelo
        model = sm.OLS(Y, X,
                       missing = 'drop').fit()
        print(model.summary())
        print("\n")

At_bats_2_t_1
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.102
Date:                Fri, 24 Mar 2023   Prob (F-statistic):              0.294
Time:                        15:54:33   Log-Likelihood:                 95.705
No. Observations:                 592   AIC:                            -187.4
Df Residuals:                     590   BIC:                            -178.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0240      0.009      

Repitamos el mismo procedimiento para los filderos

In [181]:
# Good practice
pitcher_high_stat = sorted(pitcher_high_stat)
# Dummy
I_pitcher = sorted([stat + '_I' for stat in pitcher_regular_stats])

Verifiquemos que tengan la misma longitud

In [182]:
len(I_pitcher) == len(pitcher_high_stat) == len(pitcher_regular_stats)

True

In [183]:
# Pitcher
for stat in range(0,len(pitcher_regular_stats)):
    # Variable auxiliar
    X_pitcher = []
    
    # Variables 
    i = (-1)**(empiric_panel_pitcher[I_pitcher[stat]])
    x = empiric_panel_pitcher[pitcher_regular_stats[stat]]/np.sqrt(empiric_panel_pitcher[pitcher_high_stat[stat]])
    X_pitcher = i*x
    
    # X name
    name = 'X_' + pitcher_regular_stats[stat]
    empiric_panel_pitcher[name] = X_pitcher

In [184]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

Name: X_Saves_2_t_1
Name: X_Saves_t_1
Name: X_Wins_2_t_1
Name: X_Wins_t_1


Imputemos dichas columnas con la media

In [185]:
# calculate the mean of each column
fielder_means = empiric_panel_pitcher.mean()

# impute missing data with the mean value of the column
empiric_panel_pitcher = empiric_panel_pitcher.fillna(fielder_means)

Tenemos que borrar las columnas que terminan con '_L', '_H' o '_I'

In [186]:
# define a list of suffixes to drop
drop_suffix = ['_I', '_L', '_H']

# Use a list comprehension to filter the columns to drop
fielder_cols_to_drop = [col for col in empiric_panel_pitcher.columns if col.endswith(tuple(drop_suffix))]

# Drop the selected columns
empiric_panel_pitcher = empiric_panel_pitcher.drop(fielder_cols_to_drop,
                                                   axis=1)

In [187]:
empiric_panel_pitcher.shape

(546, 184)

In [188]:
empiric_panel_pitcher['Posicion_t'].value_counts()

No       273
RP       162
SP        99
RP/CL     11
SS         1
Name: Posicion_t, dtype: int64

In [189]:
panel_columns = empiric_panel_pitcher.columns

for name in panel_columns:
    print(name)

Acronimo_t
Acronimo_t_1
Agente_libre
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad de equipos_t
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Carreras_2_t
Carreras_2_t_1
Carreras_ganadas_2_t
Carreras_ganadas_2_t_1
Carreras_ganadas_t
Carreras_ganadas_t_1
Carreras_t
Carreras_t_1
Comando_2_t
Comando_2_t_1
Comando_t
Comando_t_1
Control_2_t
Control_2_t_1
Control_t
Control_t_1
Dominio_2_t
Dominio_2_t_1
Dominio_t
Dominio_t_1
ERA_2_t
ERA_2_t_1
ERA_t
ERA_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Equipo_t
Equipo_t_1
Estado_t
Ganancias_t
Ganancias_t_1
Inning_pitched_2_t
Inning_pitched_2_t_1
Inning_pitched_t
Inning_pitched_t_1
Juegos totales_t
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Losses_2_

In [195]:
empiric_panel_pitcher[['X_Wins_2_t_1']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   X_Wins_2_t_1  546 non-null    float64
dtypes: float64(1)
memory usage: 4.4 KB


In [198]:
# Rplace infinity values
empiric_panel_pitcher['X_Wins_2_t_1'] = empiric_panel_pitcher['X_Wins_2_t_1'].replace(np.inf, np.nan)
empiric_panel_pitcher['X_Wins_t_1'] = empiric_panel_pitcher['X_Wins_t_1'].replace(np.inf, np.nan)

In [199]:
empiric_panel_pitcher.to_csv(general_dynamic_path + 'panel_fielders_cum_ch' + '.csv',
                             index = False)

In [200]:
# Pitcher
for y_name in Y_list:
    for stat in range(0,len(pitcher_regular_stats)):
        # Stat
        print(colored(pitcher_regular_stats[stat], "cyan"))

        # X name
        name = 'X_' + pitcher_regular_stats[stat]

        # OLS variables
        Y = empiric_panel_pitcher[y_name].tolist()
        X = empiric_panel_pitcher[name].tolist()
        X = sm.add_constant(X)

        # Modelo
        model = sm.OLS(Y, X,
                       missing = 'drop').fit()
        print(model.summary())
        print("\n")

Bateos_2_t_1
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     3.103
Date:                Fri, 24 Mar 2023   Prob (F-statistic):             0.0787
Time:                        16:35:28   Log-Likelihood:                 140.65
No. Observations:                 546   AIC:                            -277.3
Df Residuals:                     544   BIC:                            -268.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005      0.008      0

In [ ]:
# Dimentions
print(empiric_panel_hitter.shape)

In [ ]:
# Dimentions
print(empiric_panel_pitcher.shape)